# 古典派の二分法と金融政策

<div name="html-admonition" style="font-size: 0.8em">
<input type="button" onclick="location.href='https://translate.google.com/translate?hl=&sl=ja&tl=en&u='+window.location;" value="Google translation" style="color:#ffffff;background-color:#008080; height:25px" onmouseover="this.style.background='#99ccff'" onmouseout="this.style.background='#008080'"/> in English or the language of your choice.
</div><br>

In [ ]:
import japanize_matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import py4macro

# 警告メッセージを非表示
import warnings
warnings.filterwarnings("ignore")

## はじめに

「古典派の二分法」とは，実質変数と名目変数は別々の異なるメカニズムで決定されているという考え方である。実質変数である実質GDPや相対価格は，技術水準，選好，市場構造，資源の保有量などの「実質的」な要因に依存し，一方，財・サービスの価格や一般物価水準はマネーストックによって決定されると考える。またこの考えから派生する結果が「貨幣の中立性」である。即ち，中央銀行によるマネーストックの調整を通した金融政策は実質変数に影響を及ぼさない。実質GDPは金融政策から独立であるという考えである。

「古典派の二分法」と「貨幣の中立性」が成立するかどうかによって，マクロ経済学の短期分析のアプローチが全くことになる。前章で議論したRBCモデルにはマネーストックが入っておらず，インフレ率も出てこない。古典派の二分法が成立し，一般物価水準とインフレ率は別のメカニズムで決定されているので，わざわざ明示的にRBCの中で議論する必要がないということである。その裏にある仮定が，瞬時に調整される価格である。この仮定が成立する限り，実質変数と名目変数の決定プロセスを同時にモデル化することは余分な労力であり，実質変数のみにフォーカスすれば良いのである。

本章の目的は，「古典派の二分法」と「貨幣の中立性」が成立するかをデータに基づき考察することである。計量経済学の洗練された手法を用いて検証するということではなく，次の２つのデータセットを使って「古典派の二分法」と「貨幣の中立性」が成立しないこと示唆する「状況証拠」を考える。
* GDPとインフレ率の相関性
* ブレトンウッズ体制の崩壊

## GDPとインフレ率

`py4macro`に含まれるデータセット`jpn-q`を使うが，その中にある`gdp`と`inflation`を使い両変数の関係を考える。

In [ ]:
df = py4macro.data('jpn-q').loc[:,['gdp','inflation']]

次の変数を作成し図示しよう。
* GDPトレンドからの乖離率（％）
* インフレ率のトレンドからの乖離（％ポイント）

In [ ]:
df['GDPの変動'] = 100*np.log( df['gdp']/py4macro.trend(df['gdp']) )
df['インフレ率の変動'] = df['inflation'] - py4macro.trend(df['inflation'])

ax_ = df[['GDPの変動','インフレ率の変動']].plot()
ax_.set_title('トレンドからの乖離（％）', size=20)
pass

この図は次の２つの点を示している。第一に，GDPとインフレ率の変動は循環的である。即ち，同じ方向に動く傾向にある。第二に，GDPの変動に後追いするようにインフレ率が動いている。この点をさらに探るためにGDPとラグがあるインフレ率の相関係数を計算してみよう。次のコードの`shift()`は以前も使ったが，変数を１期ずらすメソッドである。引数に負の整数を設定することにより先の期の変数を指定することができる。

In [ ]:
print(' '*6+'GDPの変動との相関係数\n'+'-'*30)
for n in range(12):
    
    df_temp = df.copy()   # 1
    
    df_temp['インフレ率の変動_shift'] = df_temp['インフレ率の変動'].shift(-n)
    corr = df_temp[['GDPの変動', 'インフレ率の変動_shift']].corr().iloc[0,1]
    
    print(f'{n:>3}期先のインフレ率の変動: {corr:>6.3f}')

```{admonition} コードの説明
:class: dropdown

1. `.copy()`を使い`df`のコピーを作成し，それを`df_temp`に割り当てる。続く行では`df_temp`を使い計算などを行うが，１回のループが終わる度に`df_temp`は`df`のコピーとしてリセットされる。
```

実質GDPとインフレ率の相関係数は徐々に高くなり，２期先で最大になり減少することが確認でき，両変数は密接に関係していることを示唆している。一方で，因果関係を示すための洗練された手法を使っているのではなく，単なる相関係数であり「状況証拠」にしか過ぎない。しかし「状況証拠」も含めてデータが示しているものを説明できてこそ「真のメカニズム」を捉える経済モデルということができる。RBCモデルは古典派の二分法が成立しており，実質GDPとインフレ率の相関関係を説明できないのが大きな欠点の一つとなっている。

## 金融政策

金融政策が実質GDPのような実質変数に影響を及ぼすかという問題は，一見明白なように見えるが非常に複雑な問題である。因果関係を示すために様々なそして高度な手法が提案されており，それを説明することはこのサイトの範囲を超えることになる。ここでは代わりに，歴史的に重大な金融政策が実質変数に影響をもたらした例を取り上げる。[参考資料](https://eml.berkeley.edu/~enakamura/papers/macroempirics.pdf)

### 相対価格（おさらい）

本題に入る前に，実質変数と名目変数の違いを確認する。$w$を名目賃金（円），$p_A$をA財の価格（円）としよう。実質賃金$\dfrac{w}{p_A}$は，$w$円で購入できるA財の数量を表している。即ち，A財で測る名目賃金$w$の購買力を示しており，$w$の実質的な価値はA財の$\dfrac{w}{p_A}$個に相当するという意味である。

次にB財の価格を$p_B$として相対価格$\dfrac{p_B}{p_A}$を考えよう。実質賃金を使うと次のように解釈できる。

$$
\begin{align*}
\frac{p_B}{p_A}
&=
\frac{w/p_A}{w/p_B}=
\frac{\text{名目賃金$w$で購入できるA財の数量}}{\text{名目賃金$w$で購入できるB財の数量}} \\
&=\text{B財１単位に対してのA財の数量}
\end{align*}
$$

即ち，B財１単位はA財$\dfrac{p_B}{p_A}$単位に相当する。ここで重要なポイントは，$\dfrac{w}{p_A}$であっても$\dfrac{p_B}{p_A}$であってもA財の数量で表されており，貨幣単位（円）ではないことである。そういう意味で，相対価格は実質変数である。

実質変数を考える上で基準となる財はニューメレール財（価値基準財）と呼ばれる。実質賃金$\dfrac{w}{p_A}$の例ではA財がニューメレール財であり，より一般的には一般物価水準が使われる。

次に為替レートを考えよう。名目為替レート$e$は相対価格のように比率になっているが，もちろん名目である。円・ドルの名目為替レート（円/＄）は，1米ドルを何円と交換できるかの比率を表しており，単位は貨幣単位（円）で与えられる。一方で日米の実質為替レート$\varepsilon$は，もちろん，実質変数である。実質為替レートは次のように定義される。

* $e$：名目為替レート（円/＄）
* $P_{\text{日}}$：日本財の価格（円）
* $P_{\text{米}}$：米国財の価格（＄）
* $eP_{\text{米}}$：円で表示した米国財の価格（円）

$$
\varepsilon=\frac{eP_{\text{米}}}{P_{\text{日}}}
$$ (eq:16-real_ex)

この式は，次のように解釈できる。

$$
\begin{align*}
\varepsilon
&=\frac{w/P_{\text{日}}}{w/\left(eP_{\text{米}}\right)}
=\frac{\text{名目賃金$w$で購入できる日本財の数量}}{\text{名目賃金$w$で購入できる米国財の数量}} \\
&=\text{米国財１単位に対しての日本財の数量} \\
&=\text{米国財１単位に対しての日本財の交換比率}
\end{align*}
$$

実質為替レートは，米国財１単位は日本財$\varepsilon$単位に相当することを示している。重要な点は，上で説明した相対価格と同じように実質為替レートは財の数量で表された実質変数だということである。

古典派の二分法が成立している場合，相対価格は実質的な要因によって決定され，分母・分子の名目変数はマネーストックのような貨幣的な要因だけに依存することになる。この点を念頭にブレトンウッズ体制崩壊の効果について考えよう。

### ブレトンウッズ体制

歴史的な金融政策の例としてブレトンウッズ体制を考える。ブレトンウッズ体制下では名目為替レートの固定相場制が採用されていた。しかし1973年2月にブレトンウッズ体制は崩壊し、変動相場制に移行する。
* 1971年8月15日に米国はドルと金との交換停止を発表
* 1973年2月14日に変動相場制に移行

この政策変更は名目為替レートに関しての政策であり，実質的な要因の変化ではない。純粋に貨幣的な政策変更である。まず，この金融政策変更の効果を思考実験として考えるために，全ての財・サービスの価格は十分に伸縮的であり，瞬時に価格は調整され市場は常に精算する（需要と供給は常に等しい）と**仮定**しよう。この仮定のもとでは，古典派の二分法が成立し実質変数である相対価格と実質為替レートは技術や選好などの「実質的」な要因のみに依存することになり，貨幣的な政策変更の影響は受けないはずである。更に，ブレトンウッズ体制は崩壊の前後では実質的な要因の大きな変化は起きていないため，実質為替レートは大きく変化していないはずである。

この事を念頭に，実質変数がどのように**変化**したかを考察する。実質為替レートの変化率は，$\dfrac{\dot{\varepsilon}}{\varepsilon}$と表すことができる。古典派の二分法の下では，$\dfrac{\dot{\varepsilon}}{\varepsilon}$は金融政策から影響を受けないことになる。言い換えると，固定相場制もしくは変動相場制であったとしても，実質変数が変化しない限り，$\dfrac{\dot{\varepsilon}}{\varepsilon}$は変わらないことになる。更に言うと，ブレトンウッズ体制崩壊の前後で実質変数の大きな変化はなかったと広く認識されているため，その時期の$\dfrac{\dot{\varepsilon}}{\varepsilon}$の動きには大きな変化は観察されないはずである。

では，実際にどうだったのだろうか。`py4macro`に含まれるデータセット`ex`を使い確かめてみよう。

### データ

In [ ]:
df_ex = py4macro.data('ex')
df_ex.head()

内容を確認しよう。

In [ ]:
py4macro.data('ex',description=1)

`.info()`を使うと行ラベルが時系列用になっていることが確認できる。

In [ ]:
df_ex.info()

`real_ex_jpus_%change`と`real_ex_geus_%change`を使い1986年3月までのデータを並べてプロットしよう。

In [ ]:
col = df_ex.columns          # 1
end_month = '1986-03-01'     # 2
float_month = pd.Timestamp('1973-02-01')  # 3

# 4
ax = df_ex.loc[:end_month,[col[1],col[0]]] \
          .plot(subplots=True, layout=(1,2),
                figsize=(10,4),legend=False)

# 5
for i, c in enumerate(['円','マルク']):
    
    # 6
    ax[0,i].axvline(float_month, c='k', linestyle='--')
    
    # 7
    ax[0,i].set_title(f'{c}/＄実質為替レート変化率 (%)', size=18)
pass

````{admonition} コードの説明
:class: dropdown

1. 列名を変数`col`に割り当てる。
2. 文字列`'1986-03-01'`を変数`end_month`に割り当てる。
3. 文字列`'1973-02-01'`を`pd.Timestamp()`関数を使いプロットの横軸に使われる変数の単位に変換し変数`float_month`に割り当てる（代わりに`pd.to_datetime()`関数を使ってもOK）。
4. 引数として`subplots=(1,2)`を使っているので２つの「軸」があり，`Numpy`の`array`として次のように格納されている。
    ```
    array([[軸１,軸２]])
    ```
    * この`array`が`ax`に割り当てられている。従って、(2)と(3)にあるように，軸１にアクセスするには`ax[0]`，軸２にアクセスするには`ax[1]`とする必要がある。
    * 列名を`col[0]`と`col[1]`を使って指定しており，`col[0]`は`col`の0番目の要素であり，`col[1]`は`col`の1番目の要素である。
    * `legend=False`は凡例を隠すため。
5. `enumerate()`は引数のリストの要素のインデックと要素自体をタプルとして返す。最初のループで`(0,'円')`，次のループで`(1,'マルク')`を返し，タプルの0番目の要素は`i`に，1番目の要素は`c`に割り当てている。
6. `axvline()`は縦線を引くメソッドであり，３つの引数を使っている。
    * `'1973-2-1'`：横軸の値
    * `c=red`：色を赤に指定（`k`は黒）
    * `linestyle='--'`：点線に指定
7. `.set_title()`は「軸」にタイトルを付けている。

次のコード同じ図（色は異なる）をプロットすることもできる。
```
fig, ax = plt.subplots(1,2, figsize=(10,4))

ax[0].plot('real_ex_jpus_%change', data=df_ex.loc[:end_month,:])
ax[1].plot('real_ex_geus_%change', data=df_ex.loc[:end_month,:], color='orange')

for i, c in enumerate(['円','マルク']):
    ax[i].axvline(float_month, c='k', linestyle='--')
    ax[i].set_title(f'{c}/＄実質為替レート変化率', size=18)
```
````

図にある黒の点線は固定相場制から変動相場制に移行した1973年2月を示している。点線の右側と左側では明らかに変動に違いがある。どの程度変動の大きさが異なるのかを調べるために、固定相場制（158ヶ月）と変動相場制（158ヶ月）での標準偏差を計算し、その**比率**を計算してみよう。

In [ ]:
ex_list = [col[1],col[0]]
country_list = ['Yen/$','Mark/$']

print('\n実質為替レート変動率の標準偏差の比率（変動相場制移行後/移行前）\n'+'-'*47)
for e, c in zip(ex_list, country_list):
    
    ex_before = df_ex.loc[:'1973-01-01', e].std()
    
    ex_after = df_ex.loc['1973-02-01':end_month, e].std()
    
    print(f'{c:<8}{ex_after/ex_before:.2f}倍')

実質為替レートの変動は3倍以上になっている。固定相場制から変動相場制への移行は貨幣的な政策変更であるが，実質変数である実質為替レートに大きな影響を与えている。従って，実質為替レートに依存する他の実質変数も影響を受けることになる。例えば，輸出入に対してのインパクトがあるだろうし，最終的には実質GDPへの影響につながっていく。古典派の二分法と貨幣の中立性が成立していないことを示す結果である。

### 結果の要因

上の結果の要因について考えてみる。実質為替レートの変化率は，式[](eq:16-real_ex)を時間微分すると次式で表すことができる。

$$
\frac{\dot{\varepsilon}}{\varepsilon}=
\frac{\dot{e}}{e}+
\frac{\dot{P}_{\text{米}}}{P_{\text{米}}}-
\frac{\dot{P}_{\text{日}}}{P_{\text{日}}}
$$ (eq:16-change_in_rex)

この式を使い，ブレトンウッズ体制崩壊の前後を考えてみよう。

1973年2月までの固定相場制の場合，名目為替レートは変化しないため$\dot{e}=0$となり，式[](eq:16-change_in_rex)は次式となる。

$$
\frac{\dot{\varepsilon}}{\varepsilon}=
\frac{\dot{P}_{\text{米}}}{P_{\text{米}}}-
\frac{\dot{P}_{\text{日}}}{P_{\text{日}}}
$$  (eq:16-change_in_rex_0)

上の図を見ると$\dfrac{\dot{\varepsilon}}{\varepsilon}$は大きく動いていない。即ち，$\dfrac{\dot{P}_{\text{米}}}{P_{\text{米}}}-\dfrac{\dot{P}_{\text{日}}}{P_{\text{日}}}$の差は大きく動いていないことを意味する。次に，1973年2月以降の変動相場制の場合を考えてみよう。

$$
\frac{\dot{\varepsilon}}{\varepsilon}=
\frac{\dot{e}}{e}+
\frac{\dot{P}_{\text{米}}}{P_{\text{米}}}-
\frac{\dot{P}_{\text{日}}}{P_{\text{日}}}
$$ (eq:16-change_in_rex_1)

ここでは，名目為替レートは変動するため，$\dfrac{\dot{e}}{e}\ne 0$となり，式[](eq:16-change_in_rex_1)と式[](eq:16-change_in_rex)は同じである。式[](eq:16-change_in_rex_0)と式[](eq:16-change_in_rex_1)の共通項は$\dfrac{\dot{P}_{\text{米}}}{P_{\text{米}}}-\dfrac{\dot{P}_{\text{日}}}{P_{\text{日}}}$だが，その動きを確認しておこう。

In [ ]:
end_month = '1986-03-01'     #1
float_month = pd.Timestamp('1973-02-01')  #2

ax = ( 100 * df_ex.loc[:end_month,'relative_p_jpus'].pct_change() ) \
                  .plot(figsize=(6,4),legend=False)  #3

ax.axvline(float_month, c='red', linestyle='--')  #4
    
#5
eqn = r'$\dfrac{\dot{P}_{\text{US}}}{P_{\text{US}}}-\dfrac{\dot{P}_{\text{JP}}}{P_{\text{JP}}}$'

ax.set_title(eqn+'（縦軸：％）', size=18)  #6
pass

```{admonition} コードの説明
:class: dropdown

* `#1`：文字列`'1986-03-01'`を変数`end_month`に割り当てる。
* `#2`：文字列`'1973-02-01'`を`pd.Timestamp()`関数を使いプロットの横軸に使われる変数の単位に変換し変数`float_month`に割り当てる（代わりに`pd.to_datetime()`関数を使ってもOK）。
* `#3`：`ax`から始まる２行について説明する。
    * 上の行は`()`で囲まれているが，その中を先に計算している。`relative_p_jpus`の変化率を`.pct_change()`で計算し，`100`を乗じて`％`表示にしている。最後の`\`は，コードは次の行に続いていることを示している。ちなみに，この場合，`\`の後にコメント（例えば，`#3`）を入れるとエラーが発生する。
    * 下の行では，上の行の`Series`を使いプロットし，軸を`ax`に割り当てている。
* `#4`：`axvline()`は縦線を引くメソッドであり，３つの引数を使っている。
    * `'1973-2-1'`：横軸の値
    * `c=red`：色を赤に指定
    * `linestyle='--'`：点線に指定
* `#5`：タイトルに[LaTeX](https://ja.wikipedia.org/wiki/LaTeX)を使い，数式を文字列として変数`eqn`に割り当てている。文字列の最初にある`r`は，`LaTeX`コードを数式に表示させるために必要となる。
* `#6`：`.set_title()`は「軸」にタイトルを付けている。
```

図を見る限り，ブレトンウッズ体制崩壊の前後で大きな違いがあるように見えない。前後で標準偏差の変化を確認するために，次式を計算してみよう。

$$
\dfrac{\dot{P}_{\text{US}}}{P_{\text{US}}}-\dfrac{\dot{P}_{\text{JP}}}{P_{\text{JP}}}\text{の標準偏差の比率　}\left(\dfrac{\text{変動相場制移行後}}{\text{移行前}}\right)
$$

In [ ]:
relative_p_before = df_ex.loc[:'1973-01-01', 'relative_p_jpus'].std()
relative_p_after = df_ex.loc['1973-02-01':end_month, 'relative_p_jpus'].std()

print(f'{relative_p_after/relative_p_before:.2f}倍')

相対価格の変化率の変動は，むしろブレトンウッズ体制崩壊後に減少していることがわかる。即ち，式[](eq:16-change_in_rex_1)を使うと，変動相場制移行後の実質為替レートの変動$\dfrac{\dot{\varepsilon}}{\varepsilon}$は，名目為替レートの変動$\dfrac{\dot{e}}{e}$の大きな影響をそのまま反映していることになる。名目変数$e$の変化率の変動が実質変数$\varepsilon$の変化率の変動として姿を変えて現れたことになる。

では，なぜ$\dfrac{\dot{P}_{\text{米}}}{P_{\text{米}}}-\dfrac{\dot{P}_{\text{日}}}{P_{\text{日}}}$は大きく変化しなかったのだろうか。理由は，**価格の粘着性**と考えられる。価格調整に時間がかかるため，$P_{\text{米}}$と$P_{\text{日}}$は大きく動かない。従って，固定相場制の下で$\dfrac{\dot{P}_{\text{米}}}{P_{\text{米}}}-\dfrac{\dot{P}_{\text{日}}}{P_{\text{日}}}$は小さな値となり，変動相場制の下でも大きく変化しなかったのである。まとめると，データは価格の粘着性により古典派の二分法は成立していないことを示している。

ブレトンウッズ体制の崩壊という歴史的な「実験」は，金融政策は実質変数に影響を与えることを示唆している。一方，前章のRBCモデルは貨幣の中立性が成立（仮定）しており，マクロ経済の短期的な動きを十分に捉えているとは言い難い。その点を克服する理論が次章で考察するニューケインジアン・モデルである。